In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing
from keras.layers.wrappers import TimeDistributed
from keras.layers import Activation, Dense
import nltk
from tensorflow.keras import preprocessing , utils
import os
import yaml
from keras.preprocessing.text import Tokenizer
import pickle

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:

dir_path = '/content/drive/My Drive/Chatbot/data'
files_list = os.listdir(dir_path + os.sep)

questions =[]
answers =[]

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])
print(len(answers))
print(len(questions))
answers_with_tags = list()
for i in range(len(answers)):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop(i)

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

620
620
VOCAB SIZE : 2326


In [ ]:
with open("/content/drive/My Drive/Chatbot/questions.txt", "wb") as fp:
  pickle.dump(questions, fp)

In [ ]:
with open("/content/drive/My Drive/Chatbot/ans.txt", "wb") as fp:
  pickle.dump(answers, fp)

In [ ]:
tokenizer = preprocessing.text.Tokenizer()

In [ ]:
questions=[]
answers=[]

In [ ]:
with open("/content/drive/My Drive/Chatbot/questions.txt", "rb") as fp:
  questions=pickle.load(fp)

In [ ]:
with open("/content/drive/My Drive/Chatbot/ans.txt", "rb") as fp:
  answers=pickle.load(fp)

In [ ]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 1920


In [ ]:

from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append( word )

def tokenize( sentences ):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub( '[^a-zA-Z]', ' ', sentence )
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append( tokens )
    return tokens_list , vocabulary

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )


(618, 22) 22
(618, 74) 74
(618, 74, 2326)


In [ ]:
maxlen_questions=22
maxlen_answers=74

In [ ]:

encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )

from keras.layers import Activation, dot, concatenate
attention = dot([decoder_embedding, encoder_embedding], axes=[2, 2])
attention = Activation('softmax')(attention)
context = dot([attention, encoder_embedding], axes=[2,1])
decoder_combined_context = concatenate([context, decoder_embedding])
outputs = TimeDistributed(Dense(64, activation="relu"))(decoder_combined_context)
outputs = TimeDistributed(Dense(VOCAB_SIZE, activation="softmax"))(outputs)


decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy',metrics='accuracy')

model.summary()


In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=150 ) 

In [ ]:
def make_inference_models():
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


In [ ]:
import re
def str_to_tokens( sentence : str ):
    sentence=re.sub(r'[^\w\s]', '',sentence) 
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


In [ ]:
enc_model , dec_model = make_inference_models()
for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ))
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    link=''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
        empty_target_seq = np.zeros((1,1))  
        empty_target_seq[0,0] = sampled_word_index
        states_values=[h,c] 
    if ('https' in decoded_translation):
      decoded_translation=decoded_translation.split('https')
      s1=decoded_translation[1].split(' ')
      s1.remove('')
      for i in range(len(s1)-2):
         s1[i]=s1[i].replace(s1[i],s1[i]+'.')
      s1.remove('end')
      s2='https://'+('').join(s1)
      print(decoded_translation[0]+s2+'.')
    else:
      print(decoded_translation[:-3]+'.' )


In [ ]:
with open(r'/content/drive/My Drive/Chatbot/encoder_model/encoder_model2.json', 'w', encoding='utf8') as f:
    f.write(enc_model.to_json())
enc_model.save_weights(r'/content/drive/My Drive/Chatbot/encoder_model/encoder_model_weights2.h5')

with open('/content/drive/My Drive/Chatbot/decoder_model/decoder_model2.json', 'w', encoding='utf8') as f:
    f.write(dec_model.to_json())
dec_model.save_weights(r'/content/drive/My Drive/Chatbot/decoder_model/decoder_model_weights2.h5')

In [ ]:
def load_model(model_filename, model_weights_filename):
    with open(model_filename, 'r', encoding='utf8') as f:
        model = tf.keras.models.model_from_json(f.read())
    model.load_weights(model_weights_filename)
    return model

In [ ]:
encoder = load_model(r'/content/drive/My Drive/Chatbot/encoder_model/encoder_model2.json', r'/content/drive/My Drive/Chatbot/encoder_model/encoder_model_weights2.h5')
decoder = load_model(r'/content/drive/My Drive/Chatbot/decoder_model/decoder_model2.json', r'/content/drive/My Drive/Chatbot/decoder_model/decoder_model_weights2.h5')

In [ ]:
enc_model = encoder
dec_model=decoder
for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ))
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True

        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ]
    if ('https' in decoded_translation):
      decoded_translation=decoded_translation.split('https')
      s1=decoded_translation[1].split(' ')
      s1.remove('')
      for i in range(len(s1)-2):
         s1[i]=s1[i].replace(s1[i],s1[i]+'.')
      s1.remove('end')
      s2='https://'+('').join(s1)
      print(decoded_translation[0]+s2+'.')
    else:
      print(decoded_translation[:-3]+'.' )